In [84]:
import os
import numpy as np
from dotenv import load_dotenv

from FlagEmbedding import BGEM3FlagModel

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores import Chroma

In [85]:
corpus_path = r"C:\Users\piyus\OneDrive\Documents\Github Repo\Inquiry-Assistant\Context\clean_text.txt"

with open(corpus_path, "r", encoding="utf-8") as f:
    full_text = f.read()

In [86]:
env_path = r"C:\Users\piyus\OneDrive\Documents\Github Repo\.env"
load_dotenv(dotenv_path=env_path)

os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')


os.environ["HUGGINGFACE_HUB_TOKEN"] = os.getenv('HUGGINGFACE_HUB_TOKEN')

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'inquiry-assistant'

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')  

In [87]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", "--- --- "]
)

documents = splitter.create_documents([full_text])

In [88]:
class BGEEmbedding:
    def __init__(self):
        self.model = BGEM3FlagModel('BAAI/bge-small-en', use_fp16=False)

    def embed_documents(self, texts):
        texts = ["passage: " + t for t in texts]
        output = self.model.encode(texts)
        dense_vecs = output["dense_vecs"]
        dense_vecs = dense_vecs / np.linalg.norm(dense_vecs, axis=1, keepdims=True)
        return dense_vecs.tolist() 
    

    def embed_query(self, text):
        output = self.model.encode(["query: " + text])
        dense_vec = output["dense_vecs"][0]
        dense_vec = dense_vec / np.linalg.norm(dense_vec)
        return dense_vec.tolist()


embedding_function = BGEEmbedding()


Fetching 13 files: 100%|██████████| 13/13 [00:00<00:00, 12881.16it/s]


In [89]:
# Step 3: Create and persist the Chroma vectorstore
persist_directory = r"C:\Users\piyus\OneDrive\Documents\Github Repo\Inquiry-Assistant\vectorstore\chroma_db"

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_function,
    persist_directory=persist_directory
)

vectorstore.persist()
print("Vectorstore created and persisted at:", persist_directory)

pre tokenize: 100%|██████████| 3/3 [00:00<00:00, 15.87it/s]
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 3/3 [00:26<00:00,  8.75s/it]


Vectorstore created and persisted at: C:\Users\piyus\OneDrive\Documents\Github Repo\Inquiry-Assistant\vectorstore\chroma_db


In [90]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [91]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key=groq_api_key, model="gemma2-9b-it")

In [92]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define the chat-style prompt template
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are an intelligent assistant for Scaler Academy, trained on internal documents, placement records, program curricula, and student feedback. "
     "You should answer questions only based on the context provided. "
     "If the answer is not found in the context, reply with: "
     "“I'm sorry, I couldn't find that information in the available documents.” "
     "Be precise, concise, and maintain a professional and helpful tone."
    ),
    ("human", 
     "Context:\n{context}\n\nQuestion:\n{question}")
])

In [93]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [94]:
from langchain_core.runnables import RunnableLambda, RunnableMap

retriever_runnable = RunnableLambda(lambda x: retriever.invoke(x["question"]))

def format_inputs(inputs):
    return {
        "context": "\n\n".join([doc.page_content for doc in inputs["documents"]]),
        "question": inputs["question"]
    }

In [95]:
rag_chain = (
    RunnableMap({
        "documents": retriever_runnable,
        "question": lambda x: x["question"]
    })
    | format_inputs | chat_prompt | llm | output_parser
)

In [96]:
query = input("Enter your question: ")
print("\nQuestion:", query)

response = rag_chain.invoke({"question": query})
print("\nAnswer:", response)


Question: what is scaler

Answer: Scaler Academy is an institution that offers data science courses. 

